In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## Partimos de los datos genereados por Notebook creacionFeatures

In [2]:
df_train = pd.read_csv('../data/train_con_features.csv', index_col='Unnamed: 0')
df_test = pd.read_csv('../data/test_con_features.csv', index_col='Unnamed: 0')

In [3]:
df_train.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'cantidad_amenities',
       'cantidad_servicios', 'relacion_metros', 'metros_por_habitacion',
       'metros_por_bano', 'metros_por_amenities', 'bano_por_habitacion',
       'garages_por_habitacion', 'precio_prom_ciudad', 'precio_prom_propiedad',
       'provincia_top5', 'es_Apartamento', 'es_Casa_en_condominio', 'es_Casa',
       'es_Terreno', 'es_Terreno_comercial', 'es_Local_Comercial',
       'es_Quinta_Vacacional', 'es_Oficina_comercial', 'es_Edificio',
       'es_Casa_uso_de_suelo', 'es_Local_en_centro_comercial',
       'es_Bodega_comercial', 'es_Otros', 'es_Villa', 'es_Duplex',
       'es_Inmuebles_productivos_urbanos', 'es_Departamento_Compar

### Eliminamos los features generados relacionados con TipoPropiedad

In [4]:
lst_tipos_propiedades = ['es_Apartamento', 'es_Casa_en_condominio', 'es_Casa', 'es_Terreno',
                         'es_Terreno_comercial', 'es_Local_Comercial', 'es_Quinta_Vacacional',
                         'es_Oficina_comercial', 'es_Edificio', 'es_Casa_uso_de_suelo',
                         'es_Local_en_centro_comercial', 'es_Bodega_comercial', 'es_Otros',
                         'es_Villa', 'es_Duplex', 'es_Inmuebles_productivos_urbanos',
                         'es_Departamento_Compartido', 'es_Nave_industrial', 'es_Rancho',
                         'es_Terreno_industrial', 'es_Huerta', 'es_Lote', 'es_Hospedaje', 'es_Garage']

df_train = df_train.drop(lst_tipos_propiedades, axis=1)
df_test = df_test.drop(lst_tipos_propiedades, axis=1)
df_train.columns

Index(['id', 'titulo', 'descripcion', 'tipodepropiedad', 'direccion', 'ciudad',
       'provincia', 'antiguedad', 'habitaciones', 'garages', 'banos',
       'metroscubiertos', 'metrostotales', 'idzona', 'lat', 'lng', 'fecha',
       'gimnasio', 'usosmultiples', 'piscina', 'escuelascercanas',
       'centroscomercialescercanos', 'precio', 'cantidad_amenities',
       'cantidad_servicios', 'relacion_metros', 'metros_por_habitacion',
       'metros_por_bano', 'metros_por_amenities', 'bano_por_habitacion',
       'garages_por_habitacion', 'precio_prom_ciudad', 'precio_prom_propiedad',
       'provincia_top5', 'seguridad_descripcion', 'moderno_descripcion',
       'hermoso_descripcion', 'estado_descripcion', 'seguridad_titulo',
       'moderno_titulo', 'hermoso_titulo', 'estado_titulo',
       'cantidad_palabras_descripcion', 'relacion_palabras_descripcion',
       'es_avenida', 'es_ciudad_capital', 'ciudad_turistica_top15', 'anio',
       'mes'],
      dtype='object')

### Eliminamos features repetidos y complejos

In [5]:
drop = ['titulo', 'descripcion', 'direccion', 'idzona', 'lat', 'lng', 'fecha']

df_train = df_train.drop(drop, axis=1)
df_test = df_test.drop(drop, axis=1)
df_train.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,moderno_titulo,hermoso_titulo,estado_titulo,cantidad_palabras_descripcion,relacion_palabras_descripcion,es_avenida,es_ciudad_capital,ciudad_turistica_top15,anio,mes
0,254099,Apartamento,Benito Juárez,Distrito Federal,8.116114,2.0,1.0,2.0,80.0,80.0,...,0,0,0,0,0.000000,1,0,0,2015,8
1,53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,10.000000,3.0,2.0,2.0,268.0,180.0,...,0,0,0,1,0.007968,1,0,0,2013,6
2,247984,Casa,Tonalá,Jalisco,5.000000,3.0,2.0,2.0,144.0,166.0,...,0,0,0,10,0.046957,0,0,0,2015,10
3,209067,Casa,Zinacantepec,Edo. de México,1.000000,2.0,1.0,1.0,63.0,67.0,...,0,0,0,0,0.000000,0,0,0,2012,3
4,185997,Apartamento,Zapopan,Jalisco,10.000000,2.0,1.0,1.0,95.0,95.0,...,0,0,0,1,0.074108,0,0,0,2016,6


In [6]:
df_train.shape

(240000, 42)

## Aplicamos encodings a los features categoricos

In [7]:
import category_encoders as ce

cat_features = ['tipodepropiedad', 'ciudad', 'provincia']
#Elejimos un metodo de encode para estos features, y comentamos el resto de los metodos

### CatBoost Encoding

In [8]:
catb_enc = ce.CatBoostEncoder(cols=cat_features)
catb_enc.fit(df_train[cat_features], df_train['precio'])

df_train = df_train.join(catb_enc.transform(df_train[cat_features]).add_suffix('_cb'))
df_test = df_test.join(catb_enc.transform(df_test[cat_features]).add_suffix('_cb'))

### Target Encoding

In [9]:
'''target_enc = ce.TargetEncoder(cols=cat_features)
target_enc.fit(df_train[cat_features], df_train['precio'])

df_train = df_train.join(target_enc.transform(df_train[cat_features]).add_suffix('_target'))
df_test = df_test.join(target_enc.transform(df_test[cat_features]).add_suffix('_target'))'''

"target_enc = ce.TargetEncoder(cols=cat_features)\ntarget_enc.fit(df_train[cat_features], df_train['precio'])\n\ndf_train = df_train.join(target_enc.transform(df_train[cat_features]).add_suffix('_target'))\ndf_test = df_test.join(target_enc.transform(df_test[cat_features]).add_suffix('_target'))"

### Count Encoding

In [10]:
'''count_enc = ce.CountEncoder()
count_encoded = count_enc.fit_transform(df_train[cat_features])

df_train = df_train.join(count_encoded.add_suffix("_count"))
df_test = df_test.join(count_encoded.add_suffix("_count"))'''

'count_enc = ce.CountEncoder()\ncount_encoded = count_enc.fit_transform(df_train[cat_features])\n\ndf_train = df_train.join(count_encoded.add_suffix("_count"))\ndf_test = df_test.join(count_encoded.add_suffix("_count"))'

### Binary Encoding

In [11]:
'''binary_enc = ce.BinaryEncoder()
binary_encoded = binary_enc.fit_transform(df_train[cat_features])

df_train = df_train.join(binary_encoded.add_suffix("_binary"))
df_test = df_test.join(binary_encoded.add_suffix("_binary"))'''

'binary_enc = ce.BinaryEncoder()\nbinary_encoded = binary_enc.fit_transform(df_train[cat_features])\n\ndf_train = df_train.join(binary_encoded.add_suffix("_binary"))\ndf_test = df_test.join(binary_encoded.add_suffix("_binary"))'

In [12]:
df_train.shape

(240000, 45)

In [13]:
df_test.shape

(60000, 44)

In [14]:
#Dropeamos los features con strings ya encodeados
#Dejamos los dataset listos para modelar
df_train = df_train.drop(cat_features, axis=1)
df_test = df_test.drop(cat_features, axis=1)

In [15]:
#Pendiente: analisis feature selection

## Guardamos features

In [16]:
df_train.to_csv('../data/train_con_features_encoded.csv')
df_test.to_csv('../data/test_con_features_encoded.csv')